# Supermarket Ordering, Invoicing, and Sales

Joel Day, Nicholas Lee, and Christine Vu

Shiley-Marcos School of Engineering, University of San Diego

ADS 507: Practical Data Engineering

Professor Jonathan Sixt

February 27, 2023

***

## Data Description

### Invoices.csv

| Variable | Description  |
| --- | --- |
| Order Id | The order identification number |
| Date | The date the order was placed |
| Meal Id | The meal identification number |
| Company Id | The company identification number |
| Date of Meal | The date the meal was served |
| Participants | The number of people who participated in the meal |
| Meal Price | The cost of the meal |
| Type of Meal | The type of meal that was ordered |

### OrderLeads.csv

| Variable | Description  |
| --- | --- |
| Order Id | The order identification number |
| Company Id | The company identification number |
| Company Name | The name of the company associated with the order |
| Date | The date the order was placed |
| Order Value | The total value of the order |
| Converted | Whether or not the order was converted into a sale |

### SalesTeam.csv

| Variable | Description  |
| --- | --- |
| Sales Rep | The name of the sales representative |
| Sales Rep Id | The sales representative identification number |
| Company Name | The name of the company associated with the order |
| Company Id | The company identification number |

***

## Data Importing and Pre-processing

In [1]:
# Packages
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

import pymysql
from sqlalchemy import create_engine
import requests
import io
import os

import warnings
warnings.filterwarnings("ignore")

Import in CSV files

In [2]:
# Function to Pull Raw CSV from GitHub and Convert to Pandas Dataframe Object

def github_to_pandas(raw_git_url):
    # Pull Raw CSV File from GitHub
    file_name = str(raw_git_url)
    pull_file = requests.get(file_name).content

    # Convert Raw CSV to Pandas Dataframe
    csv_df = pd.read_csv(io.StringIO(pull_file.decode('utf-8')))

    return csv_df

In [3]:
# Pull CSV files from GitHub and Convert to Pandas Dataframe
invoice_df = github_to_pandas(
    "https://raw.githubusercontent.com/nlee98/ADS-507-Data-Engineering/main/Invoices.csv")

orderleads_df = github_to_pandas(
    "https://raw.githubusercontent.com/nlee98/ADS-507-Data-Engineering/main/OrderLeads.csv")

salesteam_df = github_to_pandas(
    "https://raw.githubusercontent.com/nlee98/ADS-507-Data-Engineering/main/SalesTeam.csv")

print("CSV Files from GitHub Loaded into Pandas DataFrames")

### Data Pre-processing

In [4]:
# Find missing values
print("- Invoice Missing Values:\n", invoice_df.isnull().sum())
print("\n- Order Leads Missing Values:\n", orderleads_df.isnull().sum())
print("\n- Sales Team Missing Values:\n", salesteam_df.isnull().sum())

- Invoice Missing Values:
 Order Id        0
Date            0
Meal Id         0
Company Id      0
Date of Meal    0
Participants    0
Meal Price      0
Type of Meal    0
dtype: int64

- Order Leads Missing Values:
 Order Id        0
Company Id      0
Company Name    0
Date            0
Order Value     0
Converted       0
dtype: int64

- Sales Team Missing Values:
 Sales Rep       0
Sales Rep Id    0
Company Name    0
Company Id      0
dtype: int64


In [5]:
# Data types of all columns
print("- Invoice Data Types:\n", invoice_df.dtypes)
print("\n- Order Leads Data Types:\n", orderleads_df.dtypes)
print("\n- Sales Team Data Types:\n", salesteam_df.dtypes)

- Invoice Data Types:
 Order Id        object
Date            object
Meal Id         object
Company Id      object
Date of Meal    object
Participants    object
Meal Price       int64
Type of Meal    object
dtype: object

- Order Leads Data Types:
 Order Id        object
Company Id      object
Company Name    object
Date            object
Order Value      int64
Converted        int64
dtype: object

- Sales Team Data Types:
 Sales Rep       object
Sales Rep Id    object
Company Name    object
Company Id      object
dtype: object


In [6]:
# Duplicated data
print("- Invoice Duplicated Values:", invoice_df.duplicated().sum())
print("- Order Leads Duplicated Values:", orderleads_df.duplicated().sum())
print("- Sales Team Duplicated Values:", salesteam_df.duplicated().sum())

- Invoice Duplicated Values: 0
- Order Leads Duplicated Values: 0
- Sales Team Duplicated Values: 0


***

## Explore CSV Files

### Invoice CSV

In [7]:
invoice_df.head(3)

,Order Id,Date,Meal Id,Company Id,Date of Meal,Participants,Meal Price,Type of Meal
0,839FKFW2LLX4LMBB,27-05-2016,INBUX904GIHI8YBD,LJKS5NK6788CYMUU,2016-05-31 07:00:00+02:00,['David Bishop'],469,Breakfast
1,97OX39BGVMHODLJM,27-09-2018,J0MMOOPP709DIDIE,LJKS5NK6788CYMUU,2018-10-01 20:00:00+02:00,['David Bishop'],22,Dinner
2,041ORQM5OIHTIU6L,24-08-2014,E4UJLQNCI16UX5CS,LJKS5NK6788CYMUU,2014-08-23 14:00:00+02:00,['Karen Stansell'],314,Lunch


#### Transformations
* Add Underscores to each column name
* Transform Date and Date of Meal to date/datetime data types
* Time of day column
* Number of participants column

In [8]:
# Replace spaces with underscores in all dataframe column names
invoice_df.columns = invoice_df.columns.str.replace(" ", "_")
orderleads_df.columns = orderleads_df.columns.str.replace(" ", "_")
salesteam_df.columns = salesteam_df.columns.str.replace(" ", "_")

In [9]:
# Date to Date ("d-m-Y")
invoice_df["Date"] = pd.to_datetime(
    invoice_df["Date"], format='%d-%m-%Y')

In [10]:
# Drop "+HH:MM:SS" to make all uniform to UTC timezone
invoice_df["Date_of_Meal"] = invoice_df["Date_of_Meal"].apply(
    lambda x: x.split("+")[0]
)

# Convert Date_of_Meal to Datetime format
invoice_df["Date_of_Meal"] = pd.to_datetime(
    invoice_df["Date_of_Meal"],
    format = "%Y-%m-%d %H:%M:%S"
)

In [11]:
# Convert Date_of_Meal to Datetime format
invoice_df["Date_of_Meal"] = pd.to_datetime(
    invoice_df["Date_of_Meal"],
    format = "%Y-%m-%d %H:%M:%S"
)

In [12]:
# Function defining hour of the day with the time of day
def time_of_day(x):
    day_hour = x.hour
    if (day_hour >= 5) and (day_hour <= 8): # 5am - 8am
        return "Early Morning"
    elif (day_hour > 8) and (day_hour <= 12): # 9am - 12pm
        return "Late Morning"
    elif (day_hour > 12) and (day_hour <= 15): # 1pm - 3pm
        return "Early Afternoon"
    elif (day_hour > 15) and (day_hour <= 19): # 4pm - 7pm
        return "Evening"
    elif (day_hour > 19) and (day_hour <= 23): # 8pm - 11pm
        return "Night"
    else: # 12am - 4am
        return "Late Night"

In [13]:
# Apply time_of_day function to Date_of_Meal column
invoice_df["Part_of_Day"] = invoice_df["Date_of_Meal"].apply(time_of_day)

In [14]:
# Add a field to count the number of participants
invoice_df['Number_of_Participants'] = invoice_df['Participants'].apply(lambda x: x.count("'")/2)
invoice_df['Number_of_Participants'] = invoice_df['Number_of_Participants'].astype(int)

invoice_df.head(5)

,Order_Id,Date,Meal_Id,Company_Id,Date_of_Meal,Participants,Meal_Price,Type_of_Meal,Part_of_Day,Number_of_Participants
0,839FKFW2LLX4LMBB,2016-05-27,INBUX904GIHI8YBD,LJKS5NK6788CYMUU,2016-05-31 07:00:00,['David Bishop'],469,Breakfast,Early Morning,1
1,97OX39BGVMHODLJM,2018-09-27,J0MMOOPP709DIDIE,LJKS5NK6788CYMUU,2018-10-01 20:00:00,['David Bishop'],22,Dinner,Night,1
2,041ORQM5OIHTIU6L,2014-08-24,E4UJLQNCI16UX5CS,LJKS5NK6788CYMUU,2014-08-23 14:00:00,['Karen Stansell'],314,Lunch,Early Afternoon,1
3,YT796QI18WNGZ7ZJ,2014-04-12,C9SDFHF7553BE247,LJKS5NK6788CYMUU,2014-04-07 21:00:00,['Addie Patino'],438,Dinner,Night,1
4,6YLROQT27B6HRF4E,2015-07-28,48EQXS6IHYNZDDZ5,LJKS5NK6788CYMUU,2015-07-27 14:00:00,['Addie Patino' 'Susan Guerrero'],690,Lunch,Early Afternoon,2


### Customer-Order Table
Connect the customer id to each order id the customer placed. This table will link the customer information to the invoice information.

In [15]:
# Find all the occurrences of customer names then explode to convert values in lists to rows
cust = invoice_df['Participants'].str.findall(r"'(.*?)'").explode()

# Join with order id 
cust_order_df = invoice_df[['Order_Id']].join(cust)

# Factorize to encode the unique values in participants
cust_order_df['Customer_Id'] = cust_order_df['Participants'].factorize()[0] + 1
cust_order_df["Customer_Id"] = cust_order_df["Customer_Id"].astype(str)

# Rename Participants Column
cust_order_df.columns = ["Order_Id", "Participant_Name", "Customer_Id"]

# Add Last Updated Date
cust_order_df["Last_Updated"] = dt.date.today()

cust_order_df.head(9)

,Order_Id,Participant_Name,Customer_Id,Last_Updated
0,839FKFW2LLX4LMBB,David Bishop,1,2023-02-15
1,97OX39BGVMHODLJM,David Bishop,1,2023-02-15
2,041ORQM5OIHTIU6L,Karen Stansell,2,2023-02-15
3,YT796QI18WNGZ7ZJ,Addie Patino,3,2023-02-15
4,6YLROQT27B6HRF4E,Addie Patino,3,2023-02-15
4,6YLROQT27B6HRF4E,Susan Guerrero,4,2023-02-15
5,AT0R4DFYYAFOC88Q,David Bishop,1,2023-02-15
5,AT0R4DFYYAFOC88Q,Susan Guerrero,4,2023-02-15
5,AT0R4DFYYAFOC88Q,Karen Stansell,2,2023-02-15


### Order Leads CSV
* Converted Column - Whether or not a order was converted into a sale

In [16]:
orderleads_df.head(3)

,Order_Id,Company_Id,Company_Name,Date,Order_Value,Converted
0,80EYLOKP9E762WKG,LJKS5NK6788CYMUU,Chimera-Chasing Casbah,18-02-2017,4875,1
1,TLEXR1HZWTUTBHPB,LJKS5NK6788CYMUU,Chimera-Chasing Casbah,30-07-2015,8425,0
2,839FKFW2LLX4LMBB,LJKS5NK6788CYMUU,Chimera-Chasing Casbah,27-05-2016,4837,0


In [17]:
orderleads_df["Date"] = pd.to_datetime(orderleads_df["Date"])

### Sales Team CSV

In [18]:
salesteam_df.head(3)

,Sales_Rep,Sales_Rep_Id,Company_Name,Company_Id
0,Jessie Mcallister,97UNNAT790E0WM4N,Chimera-Chasing Casbah,LJKS5NK6788CYMUU
1,Jessie Mcallister,97UNNAT790E0WM4N,Tangential Sheds,36MFTZOYMTAJP1RK
2,Jessie Mcallister,97UNNAT790E0WM4N,Two-Mile Grab,H3JRC7XX7WJAD4ZO


***

## Connection to MySQL Server

In [19]:
# Manually Login to MySQL
mysql_username = str(input("Enter MySQL Username: "))
mysql_password = str(input("Enter MySQL Password: "))

mysql_conn = pymysql.connect(
    host = "localhost",
    port = int(3306),
    user = mysql_username,
    passwd = mysql_password
)

### Create Supermarket Database
* Tries to drop the database, if it previously existed
    - Otherwise, creates the database

In [20]:
# Create ADS-507_Supermarket MySQL Database
## Drop the database to create an updated version if it exists
try :
    mysql_conn.cursor().execute(
        """
        DROP DATABASE ADS_507_Supermarket;
        """
    )
    mysql_conn.cursor().execute(
        """
        CREATE DATABASE IF NOT EXISTS ADS_507_Supermarket;
        """
    )
# Create the database if it has not done so before
except: 
    mysql_conn.cursor().execute(
        """
        CREATE DATABASE IF NOT EXISTS ADS_507_Supermarket;
        """
    )

# Navigate to Supermarket Database
mysql_conn.select_db("ADS_507_Supermarket")

print("ADS-507 Supermarket Database Created")

ADS-507 Supermarket Database Created


In [21]:
pd.read_sql("SHOW TABLES", mysql_conn)

,Tables_in_ads_507_supermarket


In [22]:
create_order_table = """
    CREATE TABLE IF NOT EXISTS orders(
        Order_Id VARCHAR(100) NOT NULL,
        Company_Id VARCHAR(100) NOT NULL,
        Company_Name VARCHAR(255),
        Date DATE,
        Order_Value SMALLINT,
        Converted TINYINT UNSIGNED,
        PRIMARY KEY (Order_Id, Company_Id),
        INDEX (Company_Id),
        INDEX (Order_Id)
    )
;
"""

mysql_conn.cursor().execute(create_order_table);
print("Orders Table Created")

Orders Table Created


In [28]:
create_invoice_table = """
CREATE TABLE IF NOT EXISTS invoice (
    Order_Id VARCHAR(100) NOT NULL,
    Date DATE NOT NULL,
    Meal_Id VARCHAR(100) NOT NULL, 
    Company_Id VARCHAR(100) NOT NULL,
    Date_of_Meal DATETIME NOT NULL,
    Participants VARCHAR(255),
    Meal_Price SMALLINT,
    Type_of_Meal ENUM('Breakfast', 'Lunch', 'Dinner'),
    Part_of_Day ENUM('Early Morning', 'Late Morning', 'Early Afternoon', 'Night', 'Late Night'),
    Number_of_Participants TINYINT,
    PRIMARY KEY (Order_Id, Date),
    FOREIGN KEY (Order_Id) REFERENCES orders(Order_Id),
    FOREIGN KEY (Company_Id) REFERENCES orders(Company_Id),
    INDEX (Date),
    UNIQUE INDEX unique_order_id (Order_Id)
    )
;
"""

mysql_conn.cursor().execute(create_invoice_table);
print("Invoice Table Created")

Invoice Table Created


In [29]:
# Foreign key added on company_id to link the salesteam to the orders table
create_salesteam_table = """
    CREATE TABLE IF NOT EXISTS salesteam(
        Sales_Rep VARCHAR(255),
        Sales_Rep_Id VARCHAR(100),
        Company_Name VARCHAR(255),
        Company_Id VARCHAR(100),
        FOREIGN KEY (Company_Id) REFERENCES orders(Company_Id),
        INDEX (Sales_Rep),
        INDEX (Company_Name)
    )
"""

mysql_conn.cursor().execute(create_salesteam_table);
print("Sales Table Created")

Sales Table Created


In [30]:
create_customerorder_table = """
    CREATE TABLE IF NOT EXISTS customer_order(
        Order_Id VARCHAR(100),
        Participant_Name VARCHAR(255),
        Customer_Id VARCHAR(255),
        Last_Updated DATE,
        FOREIGN KEY (Order_Id) REFERENCES orders(Order_Id),
        INDEX (Participant_Name)
    )
"""

mysql_conn.cursor().execute(create_customerorder_table);
print("Customer_Order Table Created")

Customer_Order Table Created


In [31]:
# Create Engine to write to SQL table
engine = create_engine(
    "mysql+pymysql://{user}:{pw}@{host}/{db}".format(
        host = "localhost", 
        db = "ADS_507_Supermarket", 
        user = mysql_username, 
        pw = mysql_password))

## Load Dataframes as Tables into MySQL
* Orders
* Invoice
* Sales Lead
* Customer (cust_order_df)

In [33]:
# Load orders datafraome to SQL table
for i, df_row in orderleads_df.iterrows():
    row_value = """
    INSERT INTO ADS_507_Supermarket.orders VALUES (
        %s, %s, %s, %s, %s, %s)
        """
    mysql_conn.cursor().execute(row_value, tuple(df_row))

print("Successfully added data to orders table")

Successfully added data to orders table


In [34]:
# Load invoice dataframe to SQL table
for i, df_row in invoice_df.iterrows():
    row_value = """
    INSERT INTO ADS_507_Supermarket.invoice VALUES (
        %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s)
        """
    mysql_conn.cursor().execute(row_value, tuple(df_row))

print("Successfully added data to invoice table")

Successfully added data to invoice table


In [35]:
# Load salesteam datafraome to SQL table
for i, df_row in salesteam_df.iterrows():
    row_value = """
    INSERT INTO ADS_507_Supermarket.salesteam VALUES (
        %s, %s, %s, %s)
        """
    mysql_conn.cursor().execute(row_value, tuple(df_row))

print("Successfully added data to salesteam table")

Successfully added data to salesteam table


In [36]:
# Load customer_order datafraome to SQL table
for i, df_row in cust_order_df.iterrows():
    row_value = """
    INSERT INTO ADS_507_Supermarket.customer_order VALUES (
        %s, %s, %s, %s)
        """
    mysql_conn.cursor().execute(row_value, tuple(df_row))

print("Successfully added data to customer_order table")

Successfully added data to customer_order table


### MySQL Transformations
* **Views** (listed alphabetically):
    - Average Meal Price: Average meal price by type of meal
    - Average Participants: Average number of participants by meal type
    - Company Metrics: For each company, the total amount and average amount of each invoice monthly are shown for each meal (and displaying their respective meal). In addition, the year-to-date amount collected and yearly total are presented.
    - Customer Purchases: Customer_Name, Part_of_Day, Company_Name, Number_of_Purchases, Total_Spent
    - Customer Stats: total number of orders by each customer, total amount each customer spent, and the average amount each spent
    - Difference Days: Difference in days between the date of meal and date the order was placed
    - Percent Converted: Shows the number of orders for every company and the total converted (as a sum and proportion) and not converted to an order, as a sum.
    - Sales by Year: Number of invoices each year
    - Sales Rep Performance: Sales_Rep, Sales_Rep_Id, Company_Name, Company_Id, Profit_by_Sales_Rep
    - Total Sales: Total sales by type of meal price for each year

In [37]:
customer_stats = """
    CREATE VIEW customer_stats
        (customer_name, number_of_order, total_spent, average_spent)
        AS
        SELECT co.Participant_Name , COUNT(*), SUM(i.Meal_Price), AVG(i.Meal_Price)
        FROM customer_order AS co
            INNER JOIN invoice AS i
                ON co.Order_Id = i.Order_Id
        GROUP BY co.Participant_Name;
"""

mysql_conn.cursor().execute(customer_stats)
print("customer_stats view created")

customer_stats view created


In [38]:
pd.read_sql("SELECT * FROM customer_stats LIMIT 3;", mysql_conn)

,customer_name,number_of_order,total_spent,average_spent
0,Matthew Jackson,15,6801.0,453.4
1,Barbara Tanaka,3,189.0,63.0
2,Amy Bufford,10,3610.0,361.0


In [39]:
company_metrics = """
    CREATE VIEW company_metrics
        (company_name, year, month, 
        meal_typ_sale, monthly_total,
        monthly_average, number_of_sales, year_to_date, yearly_total)
        AS
        SELECT o.Company_Name, YEAR(i.Date) AS Year, MONTHNAME(i.Date) AS Month,
            i.Type_of_Meal,
            SUM(i.Meal_Price) AS Monthly_Total,
            AVG(i.Meal_Price) AS Monthly_Average,
            COUNT(*) AS Number_of_Sales,
            SUM(SUM(i.Meal_Price))
                OVER (PARTITION BY o.Company_Name, Year(i.Date)
                        ORDER BY MONTH(STR_TO_DATE(month, '%M'))
                        ROWS UNBOUNDED PRECEDING) AS Year_to_Date,
            SUM(i.Meal_Price)
                OVER (PARTITION BY o.Company_Name, YEAR(i.Date)) AS Yearly_Total
        FROM orders AS o
            INNER JOIN invoice AS i
                ON o.Order_Id = i.Order_Id
        GROUP BY 1, 2, 3
        ORDER BY 1, 2, MONTH(STR_TO_DATE(month, '%M'))
"""

mysql_conn.cursor().execute(company_metrics)
print("company_metrics view created")

company_metrics view created


In [40]:
pd.read_sql("SELECT * FROM company_metrics LIMIT 3;", mysql_conn)

,company_name,year,month,meal_typ_sale,monthly_total,monthly_average,number_of_sales,year_to_date,yearly_total
0,'48 Wills,2014,February,Dinner,789.0,789.0,1,789.0,1173.0
1,'48 Wills,2014,March,Dinner,384.0,384.0,1,1173.0,1173.0
2,'48 Wills,2017,February,Dinner,216.0,216.0,1,216.0,216.0


In [41]:
sales_rep_performance = """
    CREATE VIEW sales_rep_performance
        (Sales_Rep, Sales_Rep_Id, Company_Name,
        Company_Id, Profit_by_Sales_Rep,
        Min_Sale, Max_Sale)
        AS
        SELECT s.Sales_Rep AS Sales_Rep, 
            s.Sales_Rep_Id AS Sales_Rep_Id,
            s.Company_Name AS Company_Name,
            s.Company_Id AS Company_Id,
            SUM(i.Meal_Price) AS Profit_by_Sales_Rep,
            MIN(i.Meal_Price) AS Min_Sale,
            MAX(i.Meal_Price) AS Max_Sale
        FROM salesteam AS s
            INNER JOIN orders AS o
                ON s.Company_Id = o.Company_Id
            INNER JOIN invoice AS i
                ON o.Order_Id = i.Order_Id
        GROUP BY s.Sales_Rep, s.Company_Name
"""

mysql_conn.cursor().execute(sales_rep_performance)
print("sales_rep_performance view created")

sales_rep_performance view created


In [42]:
pd.read_sql("SELECT * FROM sales_rep_performance LIMIT 3;", mysql_conn)

,Sales_Rep,Sales_Rep_Id,Company_Name,Company_Id,Profit_by_Sales_Rep,Min_Sale,Max_Sale
0,Jessie Mcallister,97UNNAT790E0WM4N,Chimera-Chasing Casbah,LJKS5NK6788CYMUU,2128.0,14,690
1,Jessie Mcallister,97UNNAT790E0WM4N,Tangential Sheds,36MFTZOYMTAJP1RK,6378.0,27,829
2,Jessie Mcallister,97UNNAT790E0WM4N,Two-Mile Grab,H3JRC7XX7WJAD4ZO,3818.0,98,664


In [43]:
customer_purchases = """
    CREATE VIEW customer_purchases
        (Customer_Name, Meal_Type, Part_of_Day, Company_Name,
        Number_of_Purchases, Total_Spent, Avg_Spent_per_Meal)
        AS
        SELECT c.Participant_Name, i.Type_of_Meal, i.Part_of_Day,
            s.Company_Name, COUNT(i.Order_Id) AS Num_Purchases,
            SUM(i.Meal_Price) AS Meal_Price, 
            AVG(i.Meal_Price) AS Avg_Spent
        FROM invoice AS i
            INNER JOIN customer_order AS c
                ON i.Order_Id = c.Order_Id
            INNER JOIN salesteam AS s
                ON s.Company_Id = i.Company_Id
        GROUP BY 1,2
"""

mysql_conn.cursor().execute(customer_purchases)
print("customer_purchases view created")

customer_purchases view created


In [44]:
pd.read_sql("SELECT * FROM customer_purchases LIMIT 3;", mysql_conn)

,Customer_Name,Meal_Typ,Part_of_Day,Company_Name,Number_of_Purchases,Total_Spent,Avg_Spent_per_Meal
0,Karen Stansell,Lunch,Early Afternoon,Chimera-Chasing Casbah,1,314.0,314.0000
1,Susan Guerrero,Dinner,Night,Chimera-Chasing Casbah,2,195.0,97.5000
2,David Bishop,Dinner,Night,Chimera-Chasing Casbah,3,217.0,72.3333


In [45]:
sales_by_year = """
    CREATE VIEW sales_by_year
        AS
    SELECT YEAR(Date) AS Year, COUNT(*) AS Total_Invoices 
    FROM invoice 
    GROUP BY Year;
"""
mysql_conn.cursor().execute(sales_by_year);

In [46]:
pd.read_sql("SELECT * FROM sales_by_year", mysql_conn)

,Year,Total_Invoices
0,2014,10069
1,2015,10039
2,2016,9857
3,2017,9937
4,2018,10115


In [47]:
percent_converted = """
    CREATE VIEW percent_converted
        AS
    SELECT Company_Name, 
        COUNT(Converted) AS Converted_Total,
        SUM(IF(Converted = '1', Converted, 0)) AS Converted_to_Order,
        (COUNT(Converted) - SUM(IF(Converted = '1', Converted, 0))) AS Not_Converted,
        ROUND(((SUM(Converted)/Count(*))*100), 2) AS Percent_Converted
    FROM orders 
    GROUP BY Company_Name;
"""
mysql_conn.cursor().execute(percent_converted);

In [48]:
test_query = """
    SELECT *
    FROM percent_converted
    ORDER BY Percent_Converted DESC
    LIMIT 5;
"""

pd.read_sql(test_query, mysql_conn)

,Company_Name,Converted_Total,Converted_to_Order,Not_Converted,Percent_Converted
0,Flashlight-Type Escape,13,7.0,6.0,53.85
1,Nonparticulate Earthmoving,13,7.0,6.0,53.85
2,Uninterrupted Dining,15,8.0,7.0,53.33
3,Free-Burning Luke'S,17,9.0,8.0,52.94
4,Covert Hint,12,6.0,6.0,50.00


In [49]:
# avg_meal_price: Average meal price
avg_meal_price = """
    CREATE VIEW avg_meal_price 
    AS
        SELECT Type_of_Meal,
            ROUND(AVG(Meal_Price),2) as Average_Meal_Price
        FROM invoice
        GROUP BY Type_of_Meal;
    """
                        
mysql_conn.cursor().execute(avg_meal_price);

In [50]:
test_query = """
    SELECT * 
    FROM avg_meal_price
    ORDER BY Average_Meal_Price DESC;
    """

pd.read_sql(test_query, mysql_conn)

,Type_of_Meal,Average_Meal_Price
0,Lunch,405.09
1,Breakfast,404.63
2,Dinner,400.07


In [51]:
# total_sales: Total sales by type of meal price by year
total_sales = """
    CREATE VIEW total_sales 
    AS
        SELECT YEAR(Date) AS Year, 
            Type_of_Meal, 
            SUM(Meal_Price) as Total_Sales 
        FROM invoice
        GROUP BY Year, Type_of_Meal 
        ORDER BY Year ASC, 
        FIELD(Type_of_Meal,'Breakfast','Lunch','Dinner')
    ;
    """
                        
mysql_conn.cursor().execute(total_sales);

In [52]:
test_query = """SELECT * FROM total_sales;"""
pd.read_sql(test_query, mysql_conn)

,Year,Type_of_Meal,Total_Sales
0,2014,Breakfast,1331066.0
1,2014,Lunch,1365269.0
2,2014,Dinner,1338395.0
3,2015,Breakfast,1360415.0
4,2015,Lunch,1336229.0
5,2015,Dinner,1335497.0
6,2016,Breakfast,1342038.0
7,2016,Lunch,1353928.0
8,2016,Dinner,1318863.0
9,2017,Breakfast,1348430.0


In [53]:
# avg_participants: Average amount of participants for each type of meal
avg_participants = """
    CREATE VIEW avg_participants 
    AS
        SELECT Type_of_Meal, 
            AVG(Number_of_Participants) as Average_Participants_Per_Meal
        FROM invoice
        GROUP BY Type_of_Meal"""
                        
mysql_conn.cursor().execute(avg_participants)

test_query = """SELECT * FROM avg_participants;"""
pd.read_sql(test_query, mysql_conn)

,Type_of_Meal,Average_Participants_Per_Meal
0,Lunch,1.6678
1,Breakfast,1.6660
2,Dinner,1.6693


In [54]:
# difference_days: Difference in days between Date_of_Meal and Date
difference_days = """
    CREATE VIEW difference_days 
    AS
        SELECT Date, 
            Date_of_Meal, 
            DATEDIFF(Date, Date_of_Meal) AS Days_Between
        FROM invoice
    ;
    """
                        
mysql_conn.cursor().execute(difference_days);

In [55]:
test_query = """
    SELECT * 
    FROM difference_days 
    LIMIT 2;
    """

pd.read_sql(test_query, mysql_conn)

,Date,Date_of_Meal,Days_Between
0,2014-03-21,2014-03-17 12:00:00,4
1,2016-04-02,2016-03-28 12:00:00,5


In [56]:
test_query = """
    show tables;
"""

pd.read_sql(test_query, mysql_conn)

,Tables_in_ads_507_supermarket
0,avg_meal_price
1,avg_participants
2,company_metrics
3,customer_order
4,customer_purchases
5,customer_stats
6,difference_days
7,invoice
8,orders
9,percent_converted


In [1]:
print("Pipeline Completed!")

Pipeline Completed!
